Some details found here:

http://elasticsearch-py.readthedocs.io/en/master/

In [5]:
import requests
from pyelasticsearch.client import ElasticSearch

In [6]:
endpoint = 'https://search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com'

In [7]:
es = ElasticSearch(endpoint, port=443)

In [14]:
es.index('contacts',
          'person',
          {'name': 'Joe Tester', 'age': 25, 'title': 'QA Master'},
           id=1)

{'_id': '1',
 '_index': 'contacts',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'person',
 '_version': 4,
 'created': False,
 'result': 'updated'}

In [15]:
docs = [{'id': 2, 'name': 'Jessica Coder', 'age': 32, 'title': 'Programmer'},
          {'id': 3, 'name': 'Freddy Tester', 'age': 29, 'title': 'Office Assistant'}]
es.bulk((es.index_op(doc, id=doc.pop('id')) for doc in docs),
          index='contacts',
          doc_type='person')

{'errors': False,
 'items': [{'index': {'_id': '2',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 3,
    'created': False,
    'result': 'updated',
    'status': 200}},
  {'index': {'_id': '3',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 3,
    'created': False,
    'result': 'updated',
    'status': 200}}],
 'took': 13}

In [16]:
es.refresh('contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

In [17]:
es.get('contacts', 'person', 3)

{'_id': '3',
 '_index': 'contacts',
 '_source': {'age': 29, 'name': 'Freddy Tester', 'title': 'Office Assistant'},
 '_type': 'person',
 '_version': 3,
 'found': True}

In [19]:
es.search('name:joe OR name:freddy', index='contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 25, 'name': 'Joe Tester', 'title': 'QA Master'},
    '_type': 'person'},
   {'_id': '3',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 29,
     'name': 'Freddy Tester',
     'title': 'Office Assistant'},
    '_type': 'person'}],
  'max_score': 0.25811607,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [27]:
es.search(index='contacts', query={"query": {"match": {"name": "Tester"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 25, 'name': 'Joe Tester', 'title': 'QA Master'},
    '_type': 'person'},
   {'_id': '3',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 29,
     'name': 'Freddy Tester',
     'title': 'Office Assistant'},
    '_type': 'person'}],
  'max_score': 0.25811607,
  'total': 2},
 'timed_out': False,
 'took': 92}

In [25]:
query = {
     'query': {
         'filtered': {
             'query': {
                 'query_string': {'query': 'name:tester'}
             },
             'filter': {
                 'range': {
                     'age': {
                         'from': 27,
                         'to': 37,
                     },
                 },
             },
         },
     },
}
es.search(index='contacts', query=query)

GET /contacts/_search [status:400 request:1.011s]


ElasticHttpError: (400, 'parsing_exception')

In [27]:
es.delete_index('contacts')

{u'acknowledged': True}

In [ ]:
import elasticsearch

host = 'YOURHOST.us-east-1.es.amazonaws.com'
awsauth = AWS4Auth(your_access_key, your_secret_key, region, 'es')

es = elasticsearch.Elasticsearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=elasticsearch.connection.RequestsHttpConnection
)
print(es.cluster.health())